In [100]:
#import libraries
import os
import pandas as pd 
import numpy as np 
import matplotlib as plt
import seaborn as sns 
import gdown
from collections import defaultdict
import requests
from bs4 import BeautifulSoup

# Functions

#### fixing '5E54' becoming '5.00E+54' issue

In [77]:
def fix_PDB_ID(df):
    for i, j in enumerate(df['PDB_ID']):
        print (j)
        if len(str(j))>4:
            if '-' in j:
                X= ''.join(j.split('-')).upper()
                df.loc[i, 'PDB_ID']= X
            else:
                if '.' in j:
                    x= str(j)
                    x1= x.split('.')[0]
                    x2= x.split('+')[1]
                    X= x1+'E'+x2
                    print (X)
                    df.loc[i, 'PDB_ID']= X
                else:
                    x= str(j)
                    x1= x.split('+')[0]
                    x2= x.split('+')[1]
                    X= x1+ x2
                    print (X)
                    df.loc[i, 'PDB_ID']= X
                    
    return (df)

#### Function to identify base pair of interest

In [81]:
def find_bp_interest(df, bp, hbonds):
    if bp.split('-')[0] != bp.split('-')[1]:
        bps= [bp, "-".join(bp.split("-")[::-1])]
    else:
        bps= [bp]
        
    df1= df[df['base_pair'].isin(bps)]
    df1.index= np.arange(0, len(df1))
    
    def extract_bp(row, hbond):
        hbond_variants = [hbond, "-".join(hbond.split("-")[::-1])]
        cols_int = [f'combined_hbond_{i}' for i in range(1, 11)]  # safer

        for col in cols_int:
            if col in row:  # check if the column exists
                for hb in hbond_variants:
                    if isinstance(row[col], str) and row[col].startswith(hb):
                        return float(row[col].split('_')[1])
        return None
    
    suffix_groups = defaultdict(list)
    for col in df1.columns:
        suffix = '_'.join(col.split('_')[-2:])
    
        if suffix.startswith('hbond'):
            print (suffix)
            suffix_groups[suffix].append(col)
    
    # Combine values for each group with matching suffix
    for suffix, cols in suffix_groups.items():
        if len(cols) == 2:  # Only combine if exactly two columns share the suffix
            col1, col2 = cols
            new_col = f'combined_{suffix}'
            df1[new_col] = df1[col2].astype(str) + '_' + df1[col1].astype(str)
            
    for hbond in hbonds:
        df1[hbond] = df1.apply(lambda row: extract_bp(row, hbond), axis=1)
    
    df2= df1.dropna(subset= hbonds) #droping examples which do not contain all the hbonds user specified
    
    df3 = df2.drop(columns=[col for col in df.columns if col.startswith('combined')]) #removing the temp columns
    
    return df3

### Functions to extract meta data from RCSB

In [97]:
def get_info(p):
    en_dic={} #this dictionary will be populated for each entity id with chain list, molecule, organism, sequence, experiment, resolution information
    filename= p+'.cif'
    url= 'https://files.rcsb.org/view/%s' %filename #this url will extract experiment and resolution information 
    url_org = 'https://www.rcsb.org/structure/%s' %p #this url will extract experiment and resolution information
    url_seq= 'https://www.rcsb.org/fasta/entry/%s/display' %p #this url will extract information comes with fasta
    
    u = requests.get(url).content
    u_org = requests.get(url_org).content
    u_seq= requests.get(url_seq).content #fasta url for getting sequence information
    
    soup= BeautifulSoup(u_org, 'lxml')
    soup_seq= BeautifulSoup(u_seq, 'lxml')
    
    all_tables= pd.read_html(url_org) #in this list of tables, each tables contain detail information of each entity
    #however, we are using these tables only to extract chain ID and segment ID
    
    
    org= soup.find_all('li', id= 'header_organism') #html contents for source organism information
    se= soup_seq.find_all('body') #html contents for fasta

    #extracting experimental method information
    u1= str(u).split('#')
    for u2 in u1:  
        if '_exptl.method' in u2:
            exp_method=  u2.split("'")[1].replace('\\', '')
    
    #print (exp_method)
    #extracting resolution information 
    for u2 in u1:
        u3= u2.replace('_reconstruction.resolution_method', '')
        u4= u3.replace("'_refine.ls_d_res_high\\'", '')
        if '_em_3d_reconstruction.resolution' in u4:
            
            resolution1= (u4.replace('\\n', '').split('_em_3d_reconstruction.resolution')[1].split('_em_3d')[0]).replace(' ', '')
            if len(resolution1)==0:
                pass
            else:
                resolution= round(float(resolution1), 1)

        if '_refine.ls_d_res_high' in u4:
            print (u4)
            resolution1= (u4.replace('\\n', '').split('_refine.ls_d_res_high')[1].split('_refine.')[0]).replace(' ', '')
            if len(resolution1)==0:
                pass
            elif resolution1 =='.':
                pass
            else: 
                resolution= round(float(resolution1), 1)
            
         


    #getting detail information from fasta url
    for i in se:
        en_det=(str(i)[13: len(str(i))-11]).split('&gt;') #list of detail for each entity

    for i, j in enumerate(en_det):
        entity_id= (j.split('|')[0]).split('_')[1] #entity ID is a number, one for each entity.
        en_dic[entity_id]=[]
        #print (entity_id)
        
        for t1, t2 in enumerate(all_tables):
            if t2.shape== (1, 4): #protein entity table shape = (5, 6), nucleic acid entity table shape= (3, 6)
                pass
            else:
                #print (t2)
                if (type(t2.columns[0]))== tuple:
                    if t2.columns[0][0].split(':')[1][1:] == str(entity_id):
                        total_chain= (t2[t2.columns[1]][0])
        
        #extracting chain_list and seg_list
        chain_list=[]#list of chain ID for each entity ID, one entity can contain multiple chains
        #all the chain in one entity is same (not sure about this fact)
        seg_list=[] #list of seg ID for each entity ID, one entity can contain multiple segment
    
        
        #print (total_chain)
        #some examplex of total chain
        #PB [auth C0], VC [auth D0]
        #A, EB [auth FB]
        #A, C, E, G, IA, C, E, G, I, K, M, O Less --->example PDB: 4RMO
        
        #seg ID and chain ID can be interchanged in some fasta files 
        #therefore, seg ID and chain ID will be always extracted from the website
        total_chain_frags= total_chain.split(',')
        #print (total_chain_frags)
        for f1, f11 in enumerate(total_chain_frags): 
            f2= f11.replace('Less', '')
            #print (f2)
            if ']' in f2:
                list_str_total_chain= list(f2.split(']'))
                for h1, h2 in enumerate(list_str_total_chain):
                    list_str_total_chain[h1]= h2.replace(',', '')
                    list_str_total_chain[h1]= h2.replace(' ', '')
                if '' in list_str_total_chain:
                    list_str_total_chain.remove('')
                else:
                    pass
                
                for h1, h2 in enumerate(list_str_total_chain):
                    hh2= h2.split('[auth')
                    chain_list.append(hh2[1])
                    seg_list.append(hh2[0])
            else: 
                chain_list.append(f2.replace(' ', ''))
                seg_list.append(f2.replace(' ', ''))

        molecule= (j.split('|')[2]) #type of molecule
    
        organism= str(j.split('|')[3]).split('\n')[0] #this expressed organism
        if organism == 'null':
            organism = 'Not available'
        
        if len(org)==0:
            org_2= 'Not available'
        else:
            #find if multiple source organism is present
            org_1= (list(str(org[0]).split('>')))
            #print (org_1)
            org_2=''
            for or1, or2 in enumerate(org_1):
                if or2.endswith('</a'):
                    #print (or2)
                    org_2 += or2[:len(or2)-3]+ ', '
            org_2= org_2[: len(org_2)-2] #this is source organism
            #print (org_2)
    
        sequence= (j.split('|')[3]).split('\n')[1] #this the RNA sequence in expre
        #print (entity_id)
        #print (chain_list)
        #print (seg_list)
        
        en_dic[entity_id].append(chain_list) #chain list
        en_dic[entity_id].append(seg_list) #segment list
        en_dic[entity_id].append(molecule) #RNA type or protein type
        en_dic[entity_id].append(organism) #expressed organism
        en_dic[entity_id].append(org_2) #source organism
        en_dic[entity_id].append(sequence) #sequence
        en_dic[entity_id].append(exp_method) #experimental method
        en_dic[entity_id].append(resolution) #resolution
        #print (sequence)
    #print (en_dic)
    #en_dic[entity_id][0] --> chain list
    #en_dic[entity_id][1] --> segment list
    #en_dic[entity_id][2] --> molecule (eg. RNA type)
    #en_dic[entity_id][3] --> expressed organism
    #en_dic[entity_id][4] --> source organism
    #en_dic[entity_id][5] --> sequence
    #en_dic[entity_id][6] --> experimental method
    #en_dic[entity_id][7] --> resolution 
    
    return (en_dic)

In [98]:
#this function will extract meta data for unique IDs (PDB_ID+ chain_ID) in the col of df
#and return the data as a dictionary where 
#keys will be the unique IDs in the col column
#values will be meta data for the corresponding unique ID
def dict_pc_id(df, col):
    unq_pc_IDs= list(set(df[col].tolist()))

    print (unq_pc_IDs)
    D_pc_info={}
    for i, j in enumerate(unq_pc_IDs):
        print ('PROBLEM HERE')
        print (j)
        print ('======================> working with this unique ID' + j)
        p= j.split('_')[0] #PDB ID
        c= j.split('_')[1] #Chain ID
        D_PDB_all= get_info(p)
        for k in D_PDB_all: #k is the entity ID
            if c in D_PDB_all[k][0]:
                print (j)
                D_pc_info[j]= D_PDB_all[k]

    return D_pc_info

In [99]:
def populate_info(df):
    df['data_ext1'] = df['PDB_ID'].astype(str) + '_' + df['chain_ID_res1'].astype(str)
    df['data_ext2'] = df['PDB_ID'].astype(str) + '_' + df['chain_ID_res2'].astype(str)
    for m, n in enumerate(['data_ext1', 'data_ext2']):
        D_pc_info= dict_pc_id(df, n)
        for i, j in enumerate(df[n]):

            print ('==========================================')
            print (j)
            chain_ID= j.split('_')[1] #this is chain ID
            print (chain_ID)
            print (D_pc_info)
            list_chain_ID1= (D_pc_info[j][0])
            list_chain_ID=[]
            for ch1, ch2 in enumerate(list_chain_ID1):
                ch3= ch2.replace(' ', '')
                list_chain_ID.append(ch3)
            print ('WORKING HERE!!!')
            print (list_chain_ID)
            chain_ID_index= list_chain_ID.index(chain_ID) #this is chain ID index in the D_pc_info, to find the segment ID the same index can be used
        
            #df.loc[i, 'Experimental_Method']= D_pc_info[j][6]
            #df.loc[i, 'Resolution_(Å)']= D_pc_info[j][7]  
            df.loc[i, 'seg_ID_res'+str(m+1)]= D_pc_info[j][1][chain_ID_index]     
            df.loc[i, 'Molecule_res'+str(m+1)]= D_pc_info[j][2]
            #df.loc[i, 'Source_Organism']= D_pc_info[j][4]
            df.loc[i, 'Organism_res'+str(m+1)]= D_pc_info[j][3]
            df.loc[i, 'Chain_length_reference_res'+str(m+1)]= len(D_pc_info[j][5])

    return df


# Data preparation

#### Step 1: importing list of base pair data from google drive

In [82]:
#import data as csv

#importing option 1: from local directory
#df_bp= pd.read_csv('/Users/jonesyy/Downloads/SROP Work/data/06_06_25/all_bps_with_all_regs_apr_2025.csv')


#importing option 2: importing using the Google drive link
#url = 'https://drive.google.com/file/d/1sL2JUIHRMAOFo7k8mjKveVF2RGcu7q-4/view?usp=drive_link' #the google drive link should give editor access 
url= 'https://drive.google.com/file/d/1I5GsOlCdIdY3iTc6wFQoTdsMWKj64vHA/view?usp=drive_link'

# Convert to a downloadable link
file_id = url.split('/d/')[1].split('/')[0]
download_url = f'https://drive.google.com/uc?id={file_id}'

# Download the file
output = 'data.csv'
gdown.download(download_url, output, quiet=False)

# Load into pandas
df_bp = pd.read_csv(output)

# Delete the downloaded CSV file
os.remove(output)

Downloading...
From: https://drive.google.com/uc?id=1I5GsOlCdIdY3iTc6wFQoTdsMWKj64vHA
To: /Users/sharear/Documents/sky/RESEARCH_&_GRAD_SCHOOL/Penn_State/identify_Homo_base_pairs/scripts/data.csv
100%|██████████| 1.39M/1.39M [00:00<00:00, 14.1MB/s]


#### Step 2: Fixing '5E54' becoming '5.00E+54' issue

In [83]:
df_bp1= fix_PDB_ID(df_bp)

124D
124D
124D
124D
124D
124D
124D
124D
176D
176D
176D
176D
176D
176D
17RA
17RA
17RA
17RA
17RA
17RA
17RA
17RA
1A1T
1A1T
1A1T
1A1T
1A1T
1A1T
1A1T
1A1T
1A3M
1A3M
1A3M
1A3M
1A3M
1A3M
1A3M
1A3M
1A3M
1A3M
1A3M
1A3M
1A3M
1A4D
1A4D
1A4D
1A4D
1A4D
1A4D
1A4D
1A4D
1A4D
1A4D
1A4D
1A4D
1A4D
1A4D
1A4D
1A4D
1A4D
1A4D
1A4D
1A4T
1A4T
1A4T
1A4T
1A4T
1A4T
1A51
1A51
1A51
1A51
1A51
1A51
1A51
1A51
1A51
1A51
1A51
1A51
1A51
1A51
1A51
1A51
1A51
1A51
1A51
1A60
1A60
1A60
1A60
1A60
1A60
1A60
1A60
1A60
1A60
1A60
1A60
1A60
1A60
1A60
1A60
1A9L
1A9L
1A9L
1A9L
1A9L
1A9L
1A9L
1A9L
1A9L
1A9L
1A9L
1A9L
1A9L
1A9L
1AC3
1AC3
1AC3
1AC3
1AC3
1AC3
1AC3
1AC3
1AC3
1AC3
1AC3
1AC3
1AFX
1AFX
1AFX
1AFX
1AJF
1AJF
1AJF
1AJF
1AJF
1AJF
1AJF
1AJF
1AJL
1AJL
1AJL
1AJL
1AJL
1AJL
1AJL
1AJT
1AJT
1AJT
1AJT
1AJT
1AJT
1AJT
1AJU
1AJU
1AJU
1AJU
1AJU
1AJU
1AJU
1AJU
1AJU
1AJU
1AJU
1AKX
1AKX
1AKX
1AKX
1AKX
1AKX
1AKX
1AKX
1AKX
1AKX
1AKX
1AKX
1AKX
1AKX
1AL5
1AL5
1AL5
1AL5
1AL5
1AL5
1AL5
1AL5
1AL5
1AL5
1AL5
1AL5
1AM0
1AM0
1AM0
1AM0
1AM0
1AM0
1ANR
1ANR


#### Step 3: Dropping null residue indexes 

In [84]:
df_bp2 = df_bp1.dropna(subset=['res_index_res1', 'res_index_res2'])
df_bp2.index= np.arange(0, len(df_bp2))

#### Step 4: Making sure icode columns will not contain null values

In [85]:
#making sure icode columns will not contain null values
df_bp2['icode_res1'] = df_bp2['icode_res1'].fillna('*')
df_bp2['icode_res1'] = df_bp2['icode_res1'].replace('nan', '*')

df_bp2['icode_res2'] = df_bp2['icode_res2'].fillna('*')
df_bp2['icode_res2'] = df_bp2['icode_res2'].replace('nan', '*')


#converting residue index into integer
df_bp2['res_index_res1'] = df_bp2['res_index_res1'].astype(float).astype(int).astype(str)
df_bp2['res_index_res2'] = df_bp2['res_index_res2'].astype(float).astype(int).astype(str)

/var/folders/wh/29t7h4rn1jg5ss_v64_j72jc0000gn/T/ipykernel_34303/703794337.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bp2['icode_res1'] = df_bp2['icode_res1'].fillna('*')
/var/folders/wh/29t7h4rn1jg5ss_v64_j72jc0000gn/T/ipykernel_34303/703794337.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bp2['icode_res1'] = df_bp2['icode_res1'].replace('nan', '*')
/var/folders/wh/29t7h4rn1jg5ss_v64_j72jc0000gn/T/ipykernel_34303/703794337.py:5: SettingWithCopyWarning: 
A value is trying to be set on a

#### Step 5: Removing any hydrogen bonds which can result into some weird unstable base pairs

In [86]:
df_bp2 = df_bp2[~df_bp2['atoms_ID_hbond_1'].str.contains("'", regex=False, na=False)]
df_bp2.index= np.arange(0, len(df_bp2))

df_bp2 = df_bp2[~df_bp2['atoms_ID_hbond_2'].str.contains("'", regex=False, na=False)]
df_bp2.index= np.arange(0, len(df_bp2))

df_bp2 = df_bp2[~df_bp2['atoms_ID_hbond_3'].str.contains("'", regex=False, na=False)]
df_bp2.index= np.arange(0, len(df_bp2))

df_bp2 = df_bp2[~df_bp2['atoms_ID_hbond_1'].str.contains("OP", regex=False, na=False)]
df_bp2.index= np.arange(0, len(df_bp2))

df_bp2 = df_bp2[~df_bp2['atoms_ID_hbond_2'].str.contains("OP", regex=False, na=False)]
df_bp2.index= np.arange(0, len(df_bp2))

df_bp2 = df_bp2[~df_bp2['atoms_ID_hbond_3'].str.contains("OP", regex=False, na=False)]
df_bp2.index= np.arange(0, len(df_bp2))

#### Step 6: (optional) Removing any base pair involved with DNA or covalently modified residues

In [87]:
df_bp2['bp_name_res']= df_bp2['res_ID_res1']+'-'+df_bp2['res_ID_res2']
df_bp3= df_bp2[(df_bp2['bp_name_res']== df_bp2['base_pair'])]
df_bp3.index= np.arange(0, len(df_bp3))

In [88]:
print (df_bp2.shape)
print (df_bp3.shape)

(8746, 35)
(7664, 35)


#### Step 7: Replacing '--' from the name column to 'undefined' 

In [89]:
df_bp3['name'] = df_bp3['name'].replace('--', 'undefined')
df_bp3['name'] = df_bp3['name'].replace('WC', 'WCF')

/var/folders/wh/29t7h4rn1jg5ss_v64_j72jc0000gn/T/ipykernel_34303/1685092079.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bp3['name'] = df_bp3['name'].replace('--', 'undefined')
/var/folders/wh/29t7h4rn1jg5ss_v64_j72jc0000gn/T/ipykernel_34303/1685092079.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bp3['name'] = df_bp3['name'].replace('WC', 'WCF')


#### Step 8: Droping atom serial number columns 

In [90]:
columns_to_drop=[]
for cols in df_bp3.columns:
    if cols.startswith('atoms_serNum'):
        columns_to_drop.append(cols)
    
df_bp4 = df_bp3.drop(columns=columns_to_drop).copy()

In [32]:
df_bp4.shape

(2792231, 40)

#### Step 9: Applying a distance cut-off for all hbonds 

In [91]:
dcols=[]
for cols in df_bp4.columns:
    if cols.startswith('distance'):
        dcols.append(cols)
df_bp4[dcols] = df_bp4[dcols].astype(float)
df_bp5 = df_bp4[~((df_bp4[dcols] > 3.4).any(axis=1))]
df_bp5[dcols] = df_bp5[dcols].astype(str)
df_bp5.index= np.arange(0, len(df_bp5))

/var/folders/wh/29t7h4rn1jg5ss_v64_j72jc0000gn/T/ipykernel_34303/3081988079.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bp5[dcols] = df_bp5[dcols].astype(str)


#### Step 10: Applying a resolution cut-off 

In [34]:
df_bp5['Resolution_(Å)'] = df_bp5['Resolution_(Å)'].astype(float)
df_bp6 = df_bp5[~(df_bp5['Resolution_(Å)'] > 3)]
df_bp6.index= np.arange(0, len(df_bp6))


/var/folders/t1/d9jqqs655cs0r3yvpzy6s6280000gn/T/ipykernel_94490/1441594956.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bp5['Resolution_(Å)'] = df_bp5['Resolution_(Å)'].astype(float)


In [3]:
df_bp.columns

Index(['PDB_ID', 'Experimental_Method', 'Resolution_(Å)', 'nt1',
       'chain_ID_res1', 'res_ID_res1', 'res_index_res1', 'icode_res1', 'nt2',
       'chain_ID_res2', 'res_ID_res2', 'res_index_res2', 'icode_res2',
       'base_pair', 'name', 'Saenger', 'LW', 'DSSR', 'bp_res',
       'distance_hbond_1', 'distance_hbond_2', 'distance_hbond_3',
       'atoms_serNum_hbond_1', 'atoms_serNum_hbond_2', 'atoms_serNum_hbond_3',
       'atoms_ID_hbond_1', 'atoms_ID_hbond_2', 'atoms_ID_hbond_3',
       'distance_hbond_4', 'atoms_serNum_hbond_4', 'atoms_ID_hbond_4',
       'distance_hbond_5', 'distance_hbond_6', 'atoms_serNum_hbond_5',
       'atoms_serNum_hbond_6', 'atoms_ID_hbond_5', 'atoms_ID_hbond_6',
       'distance_hbond_7', 'distance_hbond_8', 'atoms_serNum_hbond_7',
       'atoms_serNum_hbond_8', 'atoms_ID_hbond_7', 'atoms_ID_hbond_8',
       'distance_hbond_9', 'atoms_serNum_hbond_9', 'atoms_ID_hbond_9',
       'distance_hbond_10', 'atoms_serNum_hbond_10', 'atoms_ID_hbond_10'],
      dty

In [21]:
df_bp.shape

(3728509, 49)

In [35]:
df_bp6

,PDB_ID,Experimental_Method,Resolution_(Å),nt1,chain_ID_res1,res_ID_res1,res_index_res1,icode_res1,nt2,chain_ID_res2,...,atoms_ID_hbond_6,distance_hbond_7,distance_hbond_8,atoms_ID_hbond_7,atoms_ID_hbond_8,distance_hbond_9,atoms_ID_hbond_9,distance_hbond_10,atoms_ID_hbond_10,bp_name_res
0,157D,X-RAY DIFFRACTION,1.8,A.A5,A,A,5,*,B.U20,B,...,NaN,nan,nan,NaN,NaN,nan,NaN,nan,NaN,A-U
1,157D,X-RAY DIFFRACTION,1.8,A.A6,A,A,6,*,B.U19,B,...,NaN,nan,nan,NaN,NaN,nan,NaN,nan,NaN,A-U
2,157D,X-RAY DIFFRACTION,1.8,A.A9,A,A,9,*,B.G16,B,...,NaN,nan,nan,NaN,NaN,nan,NaN,nan,NaN,A-G
3,157D,X-RAY DIFFRACTION,1.8,A.C1,A,C,1,*,B.G24,B,...,NaN,nan,nan,NaN,NaN,nan,NaN,nan,NaN,C-G
4,157D,X-RAY DIFFRACTION,1.8,A.C11,A,C,11,*,B.G14,B,...,NaN,nan,nan,NaN,NaN,nan,NaN,nan,NaN,C-G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1403980,9QRP,X-RAY DIFFRACTION,2.7,T.G69,T,G,69,*,T.C81,T,...,NaN,nan,nan,NaN,NaN,nan,NaN,nan,NaN,G-C
1403981,9QRP,X-RAY DIFFRACTION,2.7,T.G7,T,G,7,*,T.C84,T,...,NaN,nan,nan,NaN,NaN,nan,NaN,nan,NaN,G-C
1403982,9QRP,X-RAY DIFFRACTION,2.7,T.G70,T,G,70,*,T.C80,T,...,NaN,nan,nan,NaN,NaN,nan,NaN,nan,NaN,G-C
1403983,9QRP,X-RAY DIFFRACTION,2.7,T.G71,T,G,71,*,T.C79,T,...,NaN,nan,nan,NaN,NaN,nan,NaN,nan,NaN,G-C


In [36]:
df_bp6['name'].value_counts()

name
WCF            999782
undefined      162693
Wobble         126864
rHoogsteen      45927
Sheared         19283
Imino           18158
~rHoogsteen     12286
~Wobble          7783
~Sheared         6158
Platform         2715
Calcutta         2181
Metal             155
Name: count, dtype: int64

In [39]:
df_bp6.shape

 

(1403985, 40)

# Identifying base pair of interest

In [95]:
AC_r2= find_bp_interest(df_bp6, 'A-C', ['A.N6-C.N4', 'A.N1-C.N3'])
#GG_ad= find_bp_interest(df_bp5, 'A-U', ['A.N6-U.O4', 'A.N1-U.N3'])
#AA_ad= find_bp_interest(df_bp5, 'A-A', ['A.N6-A.N6', 'A.N1-A.N1'])

hbond_1
hbond_2
hbond_3
hbond_4
hbond_1
hbond_2
hbond_3
hbond_4
hbond_5
hbond_5


/var/folders/wh/29t7h4rn1jg5ss_v64_j72jc0000gn/T/ipykernel_34303/2864412857.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[new_col] = df1[col2].astype(str) + '_' + df1[col1].astype(str)
/var/folders/wh/29t7h4rn1jg5ss_v64_j72jc0000gn/T/ipykernel_34303/2864412857.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[new_col] = df1[col2].astype(str) + '_' + df1[col1].astype(str)
/var/folders/wh/29t7h4rn1jg5ss_v64_j72jc0000gn/T/ipykernel_34303/2864412857.py:34: SettingWithCopyWarning: 
A value is t

In [96]:
AC_r2

,PDB_ID,Experimental_Method,Resolution_(Å),nt1,chain_ID_res1,res_ID_res1,res_index_res1,icode_res1,nt2,chain_ID_res2,...,distance_hbond_5,atoms_ID_hbond_5,bp_name_res,combined_hbond_1,combined_hbond_2,combined_hbond_3,combined_hbond_4,combined_hbond_5,A.N6-C.N4,A.N1-C.N3
68,2LWK,SOLUTION NMR,NaN,A.A9,A,A,9,*,A.C25,A,...,nan,NaN,A-C,A.N6-C.N3_3.326,A.N6-C.N4_2.941,A.N1-C.N3_2.891,nan_nan,nan_nan,2.941,2.891
